# Combine data and create dataframes: Video games

### Import libraries

In [1]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine
from config import username, password


### Read transformed video games data and combine it
 - Rename columsn to match database schema
 - Add unique IDs and match foreign IDs

In [2]:
VGSales_file = "VGsales_Clean.csv"
VGS = pd.read_csv(VGSales_file)
VGS.head()


,Unnamed: 0,Name,Platform,Year,Publisher,NA_Sales,Company
0,0,Wii Sports,Wii,2006.0,Nintendo,41.49,Nintendo
1,2,Mario Kart Wii,Wii,2008.0,Nintendo,15.85,Nintendo
2,3,Wii Sports Resort,Wii,2009.0,Nintendo,15.75,Nintendo
3,6,New Super Mario Bros.,DS,2006.0,Nintendo,11.38,Nintendo
4,7,Wii Play,Wii,2006.0,Nintendo,14.03,Nintendo


In [3]:
MSScore_file = "new_metacritic_df.csv"
MSS = pd.read_csv(MSScore_file)
MSS.head()


,Unnamed: 0,Name,Platform,Publisher,release_date,metascore,user_score,Company,Year
0,29,GoldenEye 007,N64,Nintendo,"Aug 25, 1997",96,9.0,Nintendo,1997
1,30,Castlevania: Symphony of the Night,PS,Konami,"Oct 2, 1997",93,9.2,Sony,1997
2,31,Sid Meier's Gettysburg!,PC,Electronic Arts,"Sep 30, 1997",92,7.4,Other,1997
3,32,Final Fantasy VII,PS,SCEA,"Sep 3, 1997",92,9.2,Sony,1997
4,33,PaRappa the Rapper,PS,SCEA,"Oct 31, 1997",92,7.5,Sony,1997


In [4]:
combined_df = pd.merge(VGS, MSS, on = ['Name', 'Platform', 'Year'])
combined_df.head()

,Unnamed: 0_x,Name,Platform,Year,Publisher_x,NA_Sales,Company_x,Unnamed: 0_y,Publisher_y,release_date,metascore,user_score,Company_y
0,6,New Super Mario Bros.,DS,2006.0,Nintendo,11.38,Nintendo,4864,Nintendo,"May 15, 2006",89,8.5,Nintendo
1,11,Mario Kart DS,DS,2005.0,Nintendo,9.81,Nintendo,3893,Nintendo,"Nov 14, 2005",91,8.7,Nintendo
2,15,Kinect Adventures!,X360,2010.0,Microsoft Game Studios,14.97,Microsoft,10026,Microsoft Game Studios,"Nov 4, 2010",61,6.3,Microsoft
3,16,Grand Theft Auto V,PS3,2013.0,Take-Two Interactive,7.01,SONY,12398,Rockstar Games,"Sep 17, 2013",97,8.3,Sony
4,17,Grand Theft Auto: San Andreas,PS2,2004.0,Take-Two Interactive,9.43,SONY,3064,Rockstar Games,"Oct 26, 2004",95,9.0,Sony


In [5]:
combined_df = combined_df.drop(columns=['Unnamed: 0_x','Unnamed: 0_y','Company_x'])
combined_df.head()

,Name,Platform,Year,Publisher_x,NA_Sales,Publisher_y,release_date,metascore,user_score,Company_y
0,New Super Mario Bros.,DS,2006.0,Nintendo,11.38,Nintendo,"May 15, 2006",89,8.5,Nintendo
1,Mario Kart DS,DS,2005.0,Nintendo,9.81,Nintendo,"Nov 14, 2005",91,8.7,Nintendo
2,Kinect Adventures!,X360,2010.0,Microsoft Game Studios,14.97,Microsoft Game Studios,"Nov 4, 2010",61,6.3,Microsoft
3,Grand Theft Auto V,PS3,2013.0,Take-Two Interactive,7.01,Rockstar Games,"Sep 17, 2013",97,8.3,Sony
4,Grand Theft Auto: San Andreas,PS2,2004.0,Take-Two Interactive,9.43,Rockstar Games,"Oct 26, 2004",95,9.0,Sony


In [6]:
video_game = combined_df[['Name', 'Publisher_y']]
video_game = video_game.rename(
    columns = {
        "Name": "game_name",
        "Publisher_y": "publisher"
    }
)

video_game = video_game.drop_duplicates(subset=['game_name', 'publisher'])
video_game = video_game.reset_index(drop=True)
video_game.index.names = ["id"]

video_game

,game_name,publisher
id,,
0,New Super Mario Bros.,Nintendo
1,Mario Kart DS,Nintendo
2,Kinect Adventures!,Microsoft Game Studios
3,Grand Theft Auto V,Rockstar Games
4,Grand Theft Auto: San Andreas,Rockstar Games
...,...,...
3613,PGA European Tour,Infogrames
3614,Planet Monsters,Titus Software
3615,Bust-A-Move 3000,Ubisoft


In [7]:
games_sales = combined_df[['Name', 'NA_Sales','Year']]
games_sales = games_sales.rename(
    columns={
        "ID": "id",
        "NA_Sales": "na_sales",
        "Year": "year"
    }
)

video_games_dict = {}
for i in video_game.index: 
     video_games_dict[video_game['game_name'][i]] = i 

games_sales["id_game"] = games_sales["Name"].map(video_games_dict)

games_sales = games_sales.drop(columns=['Name'])
games_sales = games_sales[['id_game', 'na_sales', 'year']]

games_sales.index.names = ["id"]

games_sales

,id_game,na_sales,year
id,,,
0,0,11.38,2006.0
1,1,9.81,2005.0
2,2,14.97,2010.0
3,3,7.01,2013.0
4,4,9.43,2004.0
...,...,...,...
5243,3613,0.01,2000.0
5244,3614,0.01,2001.0
5245,3615,0.01,2003.0


In [8]:
platform_catalog = pd.DataFrame(combined_df['Platform'].unique())

platform_catalog = platform_catalog.rename(columns={0: 'Platform'})

platform_catalog.loc[platform_catalog['Platform']=="3DS",'Company'] = 'Nintendo'
platform_catalog.loc[platform_catalog['Platform']=="DS",'Company'] = 'Nintendo'
platform_catalog.loc[platform_catalog['Platform']== "GB",'Company'] = 'Nintendo'
platform_catalog.loc[platform_catalog['Platform']=="GBA",'Company'] = 'Nintendo'
platform_catalog.loc[platform_catalog['Platform']=="GC",'Company'] = 'Nintendo'
platform_catalog.loc[platform_catalog['Platform']=="N64",'Company'] = 'Nintendo'
platform_catalog.loc[platform_catalog['Platform']=="PC",'Company'] = 'Microsoft'
platform_catalog.loc[platform_catalog['Platform']=="PS",'Company'] = 'SONY'
platform_catalog.loc[platform_catalog['Platform']=="PS2",'Company'] = 'SONY'
platform_catalog.loc[platform_catalog['Platform']=="PS3",'Company'] = 'SONY'
platform_catalog.loc[platform_catalog['Platform']=="PS4",'Company'] = 'SONY'
platform_catalog.loc[platform_catalog['Platform']=="PSP",'Company'] = 'SONY'
platform_catalog.loc[platform_catalog['Platform']=="PSV",'Company'] = 'SONY'
platform_catalog.loc[platform_catalog['Platform']=="Wii",'Company'] = 'Nintendo'
platform_catalog.loc[platform_catalog['Platform']=="WiiU",'Company'] = 'Nintendo'
platform_catalog.loc[platform_catalog['Platform']=="X360",'Company'] = 'Microsoft'
platform_catalog.loc[platform_catalog['Platform']=="XB",'Company'] = 'Microsoft'
platform_catalog.loc[platform_catalog['Platform']=="XOne",'Company'] = 'Microsoft'
platform_catalog['Company'].fillna('Other', inplace=True)

platform_catalog = platform_catalog.rename(
    columns = {
        "Platform": "platform_name",
        "Company": "company_name"
    }
)

platform_catalog.index.names = ["id"]

platform_catalog

,platform_name,company_name
id,,
0,DS,Nintendo
1,X360,Microsoft
2,PS3,SONY
3,PS2,SONY
4,3DS,Nintendo
5,PS4,SONY
6,PS,SONY
7,PC,Microsoft
8,N64,Nintendo


In [9]:
game_platform = combined_df[['Name','Platform','release_date','metascore','user_score']]
game_platform = game_platform.rename(
    columns={
        "release_date": "released_date",
        "user_score": "userscore"
    }
)

game_platform["id_game"] = game_platform["Name"].map(video_games_dict)

platform_dict = {}
for i in platform_catalog.index:
    platform_dict[platform_catalog['platform_name'][i]] = i

game_platform["id_platform"] = game_platform["Platform"].map(platform_dict)

game_platform = game_platform.drop(columns=['Name','Platform'])
game_platform = game_platform[['id_game', 'id_platform', 'released_date', 'metascore', 'userscore']]

game_platform = game_platform.drop_duplicates(subset=['id_game', 'id_platform'])

game_platform = game_platform.set_index(['id_game', 'id_platform'])

game_platform = game_platform.replace(to_replace = 'tbd', value=-1)
        
game_platform


,,released_date,metascore,userscore
id_game,id_platform,,,
0,0,"May 15, 2006",89,8.5
1,0,"Nov 14, 2005",91,8.7
2,1,"Nov 4, 2010",61,6.3
3,2,"Sep 17, 2013",97,8.3
4,3,"Oct 26, 2004",95,9.0
...,...,...,...,...
3613,8,"Jun 15, 2000",58,8.0
3614,11,"Dec 6, 2001",67,-1
3615,10,"Feb 12, 2003",53,-1


## Persist information in PostgreSQL: Video games data

In [10]:
rds_connection_string = f'{username}:{password}@localhost:5432/videogames'
engine = create_engine(f'postgresql+psycopg2://{rds_connection_string}')

engine.table_names()


['video_game',
 'games_sales',
 'game_platform',
 'platform_catalog',
 'stock',
 'stock_history']

In [11]:
video_game.to_sql(name='video_game', con=engine, if_exists='append', index=True) 


In [12]:
games_sales.to_sql(name='games_sales', con=engine, if_exists='append', index=True) 


In [13]:
platform_catalog.to_sql(name='platform_catalog', con=engine, if_exists='append', index=True) 


In [14]:
game_platform.to_sql(name='game_platform', con=engine, if_exists='append', index=True) 
